### Задача 3. Расширенная модель и валидация

Создайте модель Library:
– books: …
– users: …
Добавьте в модель Book поле categories: List[str] с валидацией.
Реализуйте метод total_books() -> ... для модели Library.

In [1]:
from pydantic import BaseModel, Field, EmailStr, conlist, field_validator

class Book(BaseModel):
    title: str
    author: str
    year: int = Field(..., ge=0, le=2100)
    available: bool = True
    categories: conlist(str, min_length=1)

    @field_validator("categories")
    @classmethod
    def clean_categories(cls, values: list[str]) -> list[str]:
        cleaned: list[str] = []
        for value in values:
            if not isinstance(value, str):
                raise TypeError("Категория должна быть строкой")
            s = value.strip()
            if not s:
                raise ValueError("Категория не может быть пустой")
            cleaned.append(s.lower())
        return cleaned
        
class User(BaseModel):
    name: str
    email: EmailStr
    membership_id: str

class Library(BaseModel):
    books: list[Book] = Field(default_factory=list)
    users: list[User] = Field(default_factory=list)

    def total_books(self) -> int:
        return len(self.books)
        

class BookNotFound(Exception):
    pass

class BookNotAvailable(Exception):
    pass
    
class AlreadyReturned(Exception):
    pass
    

def add_book(catalog: list[Book], book: Book) -> None:
    catalog.append(book)

def find_book(catalog: list[Book], *, title: str | None = None, author: str | None = None, year: int | None = None) -> list[Book]:
    return [book for book in catalog
            if (title is None or book.title == title)
            and (author is None or book.author == author)
            and (year is None or book.year == year)]

def is_book_borrow(catalog: list[Book], title: str) -> None:
    for book in catalog:
        if book.title == title:
            if not book.available:
                raise BookNotAvailable(f"'{title}' уже выдана")
            book.available = False
            return
    raise BookNotFound(f"'{title}' не найдена")

def return_book(catalog: list[Book], title: str) -> None:
    for book in catalog:
        if book.title == title:
            if book.available:
                raise AlreadyReturned(f"'{title}' уже доступна")
            book.available = True
            return
    raise BookNotFound(f"'{title}' не найдена")
    